In [1]:
!pip install cvzone


  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26298 sha256=e5912fc4db3aeea7fdd7a6cba168e44b428a5083d4c4d39ce98e00c012fcffaf
  Stored in directory: /root/.cache/pip/wheels/ab/36/ec/47be2d4e59dc4289e684d5b0dde54d1e72e51a614e57690e85
Successfully built cvzone


In [2]:
!pip install mediapipe


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 60.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5


#**IMPORT**

In [3]:
import os
import h5py # Cần cài đặt: pip install h5py
import math
import numpy as np # Cần cài đặt: pip install numpy
import tensorflow as tf # Cần cài đặt: pip install tensorflow
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import get_file # Đường dẫn đúng cho tf.keras >= 2.x
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG # Dùng trong môi trường IPython/Jupyter Notebook
from tensorflow.keras.utils import model_to_dot # Đường dẫn đúng. Cần cài đặt: pip install pydot và graphviz
from tensorflow.keras.utils import plot_model # Cần cài đặt: pip install pydot và graphviz
from tensorflow.keras.initializers import glorot_uniform
import matplotlib.pyplot as plt # Cần cài đặt: pip install matplotlib
from matplotlib.pyplot import imshow
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from cvzone.FaceMeshModule import FaceMeshDetector
import zipfile
import gdown
import tensorflow.keras.backend as K

K.set_image_data_format('channels_last')

# **LOAD DATA**


In [1]:
!pip install mxnet


In [6]:
import mxnet as mx
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Đường dẫn tới các file
rec_path = "/content/drive/MyDrive/Dataset/faces_webface_112x112/faces_webface_112x112/train.rec"
idx_path = "/content/drive/MyDrive/Dataset/faces_webface_112x112/faces_webface_112x112/train.idx"
lst_path = "/content/drive/MyDrive/Dataset/faces_webface_112x112/faces_webface_112x112/train.lst"

# Đọc file rec và idx
record = mx.recordio.MXIndexedRecordIO(idx_path, rec_path, 'r')

# Đọc file lst
with open(lst_path, 'r') as f:
    lines = f.readlines()

X = []
y = []
error_count = 0

for line in lines:
    parts = line.strip().split()
    if len(parts) != 3:
        continue
    try:
        idx = int(parts[0])
        label = int(parts[2])

        packed = record.read_idx(idx)
        if packed is None or len(packed) == 0:
            print(f"❌ Bỏ qua idx {idx}: buffer rỗng")
            error_count += 1
            continue

        header, img = mx.recordio.unpack(packed)
        img_arr = mx.image.imdecode(img).asnumpy()
        img_arr = cv2.resize(img_arr, (112, 112))

        X.append(img_arr)
        y.append(label)

    except Exception as e:
        print(f"❌ Lỗi ở dòng {line.strip()}: {e}")
        error_count += 1
        continue

print(f"⚠️ Tổng số ảnh bị lỗi bỏ qua: {error_count}")

# Chuyển thành numpy arrays
X = np.array(X)
y = np.array(y)

# Chia train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=42)

print(f"✅ Done!")
print(f"X_train shape: {X_train.shape}")
print(f"X_test  shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
MXNetError: Check failed: len > 0: Input cannot be an empty buffer
❌ Lỗi ở dòng 0	/raid5data/dplearn/CASIA-WebFace/0935653/004.jpg	5097: Traceback (most recent call last):
  File "../src/io/image_io.cc", line 189
MXNetError: Check failed: len > 0: Input cannot be an empty buffer
❌ Lỗi ở dòng 0	/raid5data/dplearn/CASIA-WebFace/0936298/002.jpg	5104: Traceback (most recent call last):
  File "../src/io/image_io.cc", line 189
MXNetError: Check failed: len > 0: Input cannot be an empty buffer
❌ Lỗi ở dòng 0	/raid5data/dplearn/CASIA-WebFace/0936300/008.jpg	5105: Traceback (most recent call last):
  File "../src/io/image_io.cc", line 189
MXNetError: Check failed: len > 0: Input cannot be an empty buffer
❌ Lỗi ở dòng 0	/raid5data/dplearn/CASIA-WebFace/0936493/012.jpg	5108: Traceback (most recent call last):
  File "../src/io/image_io.cc", line 189
MXNetError: Check failed: len > 0: Input cannot be an empty buffer
❌ Lỗi ở dòng 0	/raid5dat

ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

#**PREPROCESSING DATA**

In [ ]:
cols = 3
rows = 3
plt.figure(figsize=(12, 8))
for i in range(rows * cols):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(X_train[i])
    plt.title(y_train[i], fontsize=8)
    plt.axis('off')

#**IDENTITY BLOCK**

In [ ]:
def identity_block(X,f,filters,stage,block):
  conv_base_name = 'res'+str(stage)+block+'_'
  bn_base_name = 'bn'+str(stage)+block+'_'
  f1,f2,f3 = filters
  bn_axis = 3  # BN hoạt động trên trục số kênh
  X_skip_connection = X

  ## Tầng đầu tiên mục đích để giảm số chiều
  X = Conv2D(filters = f1,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_base_name+'first_component',kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=bn_axis,name=bn_base_name+'first_component')(X)
  X = Activation('relu')(X)

  ## Tầng thứ hai trích xuất dữ liệu quan trọng
  X = Conv2D(filters=f2,kernel_size=(f,f),strides=(1,1),padding='same',name=conv_base_name+'second_component',kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=bn_axis,name=bn_base_name+'second_component')(X)
  X = Activation('relu')(X)
  ## Tầng thứ ba
  X = Conv2D(filters=f3,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_base_name+'third_component',kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=bn_axis,name=bn_base_name+'third_component')(X)

  X = Add()([X,X_skip_connection])
  X = Activation('relu')(X)
  return X

#**CONVOLUTIONAL BLOCK**

In [ ]:
def convolutional_block(X,f,filters,stage,block,s=2):
  conv_base_name = 'res' + str(stage) + block + '_'
  bn_base_name = 'bn' + str(stage) + block + '_'
  f1,f2,f3 = filters
  bn_axis = 3
  X_skip_connection = X

  #Tầng đầu tiên
  X = Conv2D(f1,(1,1),strides=(s,s),padding='valid',name=conv_base_name+'first_component',kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=bn_axis,name=bn_base_name+'first_component')(X)
  X = Activation('relu')(X)
  #Tầng thứ hai
  X = Conv2D(f2,kernel_size = (f,f),strides=(1,1),padding='same',name=conv_base_name+'second_component',kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=bn_axis,name=bn_base_name+'second_component')(X)
  X = Activation('relu')(X)
  #Tầng thứ ba
  X = Conv2D(f3,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_base_name+'third_component',kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=bn_axis,name=bn_base_name+'third_component')(X)
  ##Tầng kết nối tắt
  X_skip_connection = Conv2D(f3,(1,1),strides=(s,s),padding='valid',name=conv_base_name+'merge',kernel_initializer=glorot_uniform(seed=0))(X_skip_connection)
  X_skip_connection = BatchNormalization(axis=bn_axis,name=bn_base_name+'merge')(X_skip_connection)
  X = Add()([X,X_skip_connection])
  X = Activation('relu')(X)
  return X

#**RESNET50**

In [ ]:
def ResNet50(input_shape=(64,64,3),classes=6):
  X_input = Input(input_shape)
  X = ZeroPadding2D((3,3))(X_input)
  # Stage 1
  X = Conv2D(64,(7,7),strides=(2,2),name='conv1',kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3,name='bn_conv1')(X)
  X = Activation('relu')(X)

  # Stage 2
  X = convolutional_block(X,f=3,filters=[64,64,256],stage=2,block='a',s=2)
  X = identity_block(X,f=3,filters=[64,64,256],stage=2,block='b')
  X = identity_block(X,f=3,filters=[64,64,256],stage=2,block='c')

  # Stage 3
  X = convolutional_block(X,f=3,filters=[128,128,512],stage=3,block='a',s=2)
  X = identity_block(X,f=3,filters=[128,128,512],stage=3,block='b')
  X = identity_block(X,f=3,filters=[128,128,512],stage=3,block='c')
  X = identity_block(X,f=3,filters=[128,128,512],stage=3,block='d')

  # Stage 4
  X = convolutional_block(X,f=3,filters=[256,256,1024],stage=4,block='a',s=2)
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
  # Average Pooling
  X = AveragePooling2D((2, 2), name='avg_pool')(X)

  # output layer
  X = Flatten()(X)
  X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)

  # Create model
  model = Model(inputs = X_input, outputs = X, name='ResNet50')

  return model

#**TRAIN RESNET50**

In [ ]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 20, batch_size = 32)